# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items

In [1]:
name = "ItemCF";

In [2]:
using NBInclude
@nbinclude("ItemCFBase.ipynb");

## Compute cosine correlations

In [3]:
function get_correlation_matrix_outdir(residual_alphas)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $residual_alphas"
    training = RatingsDatset64(get_residuals("training", residual_alphas))
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(eltype(R), maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
item_cf_params = [[
        cf_params(
            name = "ItemCF.$K",
            training_residuals = ["UserItemBiases"],
            validation_residuals = ["UserItemBiases"],
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(["UserItemBiases"]),
            K = K,
            λ = [1.0, 1.0, 0.0],
        ) for K in [2^4, 2^6, 2^8, 2^10, 2^12]
    ];
];

[ Debug: 20220130 18:02:22 generating similarity matrix for ["UserItemBiases"]
Progress: 100%|███████████████████████████| Time: 0:34:53 ( 1.77  s/it)


## Train models

In [ ]:
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:00:12 (14.55 ms/it)
[ Debug: 20220130 18:38:20 loss: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.461814769765018,-0.032782664452458546,0.014111308880508986,-1.2776578203501232) β: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(0.5984542479729831,-0.00286784730592304,-0.4488086291183065,4.593106813515109): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsData

Iter     Function value   Gradient norm 
     0     1.461815e+00     1.277658e+00
 * Current step size: 1.0
 * time: 0.025873899459838867
 * g(x): [-0.032782664452458546, 0.014111308880508986, -1.2776578203501232]
 * x: [1.0, 1.0, 0.0]


Progress: 100%|███████████████████████████| Time: 0:00:15 (17.50 ms/it)
[ Debug: 20220130 18:38:44 loss: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.4233290293871335,0.025797575559199877,-0.010261928452484505,0.017003606371204103) β: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(2.1981955645275493,1.6876107488644765,-1.3895721733463773,0.9204990719006015): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDat

     1     1.423017e+00     2.525386e-02
 * Current step size: 0.9862863247664605
 * time: 45.59441089630127
 * g(x): [0.02525385581793893, -0.010174188846977737, 0.017118935465432538]
 * x: [1.0323330936388675, 0.9860822090265985, 1.2601364359422496]


Progress: 100%|███████████████████████████| Time: 0:00:15 (17.60 ms/it)
[ Debug: 20220130 18:39:27 loss: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.4221116999566206,0.01977510229288983,-0.00784184681135256,0.016559207462477146) β: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(2.1129622728195963,1.6382487034248794,-1.3602896453386992,0.8952933519066019): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDatas

...

Progress: 100%|███████████████████████████| Time: 0:20:46 ( 1.45  s/it)
[ Debug: 20220131 12:39:49 loss: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.395051897653871,-0.004283578410883458,0.03283546426667588,0.026405516033759912) β: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.7173744871132555,0.9419656409387631,-1.438289174036198,0.861492520304148): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset

     4     1.382255e+00     5.019218e-02
 * Current step size: 3.3733753265670456
 * time: 19272.519526958466
 * g(x): [-0.04377955084029327, 0.050192184976641824, 0.0166897460250797]
 * x: [1.321252085981515, 1.2963700976538388, 0.2102851246930112]


Progress: 100%|███████████████████████████| Time: 0:23:32 ( 1.65  s/it)
[ Debug: 20220131 14:36:29 loss: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(1.7265136676105988,-1.7084397752690321,-0.006737500796867009,-0.01648366427245576) β: Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}}(-0.5262270389746342,-57.13894541035295,0.19127312919284714,-0.28980515370809284): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#22"{var"#19#21"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsD

     5     1.381777e+00     5.092102e-02
 * Current step size: 0.004163292838706075
 * time: 26481.26772212982
 * g(x): [-0.0472028667994053, 0.050921024896957046, 0.012519735245295052]
 * x: [1.3241688788770891, 1.296584459290561, 0.18606056202953414]


Progress:  15%|████▏                      |  ETA: 0:24:12 ( 1.99  s/it)